In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import shap
from ACME.ACME import ACME
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [2]:
#! pip install catboost

In [3]:
#! pip install shap

In [4]:
#! pip install lightgbm

In [5]:
#! pip install statwolfACME

In [6]:
#! pip install xgboost

In [7]:
from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target
dataframe = pd.DataFrame(X, columns=boston.feature_names)
dataframe['target'] = y

Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datas

In [8]:
features = dataframe.drop(columns='target').columns.tolist()
X = dataframe[features]
y = dataframe['target']

In [9]:
models = {}
models['linear_regression'] = LinearRegression().fit(X,y)
models['random_forest_regressor'] = RandomForestRegressor().fit(X,y)
models['cat_boost_regressor'] = CatBoostRegressor(verbose=False).fit(X,y)
models['svr'] = SVR().fit(X,y) 
models['xgboost'] = xgb.XGBRegressor().fit(X,y)
models['lightgbm'] = lgb.LGBMRegressor().fit(X,y)

[07:50:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
mse_full = {}
for model in models.keys():
    mse_full[model] = mean_squared_error(models[model].predict(X),y)

In [11]:
acme = {}
for model in models.keys():
    acme[model] = ACME(models[model],'target')
    acme[model] = acme[model].fit(dataframe, robust=True)

In [12]:
k = 5
top_k = {}
last_k = {}
for model in models.keys():
    top_k[model] = acme[model].feature_importance().index.tolist()[0:k]
    last_k[model] = acme[model].feature_importance().index.tolist()[-k:]

In [13]:
models_top_k = {}
models_top_k['linear_regression'] = LinearRegression().fit(dataframe[top_k['linear_regression']],y)
models_top_k['random_forest_regressor'] = RandomForestRegressor().fit(dataframe[top_k['random_forest_regressor']],y)
models_top_k['cat_boost_regressor'] = CatBoostRegressor(verbose=False).fit(dataframe[top_k['cat_boost_regressor']],y)
models_top_k['svr'] = SVR().fit(dataframe[top_k['svr']],y)
models_top_k['xgboost'] = xgb.XGBRegressor().fit(dataframe[top_k['xgboost']],y)
models_top_k['lightgbm'] = lgb.LGBMRegressor().fit(dataframe[top_k['lightgbm']],y)

[07:50:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
mse_top_k = {}
for model in models.keys():
    mse_top_k[model] = mean_squared_error(models_top_k[model].predict(dataframe[top_k[model]]),y)

In [15]:
models_last_k = {}
models_last_k['linear_regression'] = LinearRegression().fit(dataframe.drop(columns = top_k['linear_regression'] + ['target']),y)
models_last_k['random_forest_regressor'] = RandomForestRegressor().fit(dataframe.drop(columns = top_k['random_forest_regressor']+ ['target']),y)
models_last_k['cat_boost_regressor'] = CatBoostRegressor(verbose=False).fit(dataframe.drop(columns = top_k['cat_boost_regressor']+ ['target']),y)
models_last_k['svr'] = SVR().fit(dataframe.drop(columns = top_k['svr']+ ['target']),y)
models_last_k['xgboost'] = xgb.XGBRegressor().fit(dataframe.drop(columns = top_k['xgboost']+ ['target']),y)
models_last_k['lightgbm'] = lgb.LGBMRegressor().fit(dataframe.drop(columns = top_k['lightgbm'] + ['target']),y)

[07:50:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
mse_last_k = {}
for model in models.keys():
    mse_last_k[model] = mean_squared_error(models_last_k[model].predict(dataframe.drop(columns = top_k[model]+ ['target'])),y)

In [17]:
mse_full

{'linear_regression': 21.894831181729202,
 'random_forest_regressor': 1.477642837944662,
 'cat_boost_regressor': 0.3362474873976234,
 'svr': 66.81823779202165,
 'xgboost': 2.366729052275626,
 'lightgbm': 1.542319288669038}

In [18]:
mse_top_k

{'linear_regression': 26.099816903706003,
 'random_forest_regressor': 2.0337227015810266,
 'cat_boost_regressor': 0.8634103986654711,
 'svr': 66.50031825228734,
 'xgboost': 4.094807811132808,
 'lightgbm': 2.660220295152345}

In [19]:
mse_last_k

{'linear_regression': 55.155023598342325,
 'random_forest_regressor': 4.172156482213433,
 'cat_boost_regressor': 3.802887869326224,
 'svr': 55.163510150619736,
 'xgboost': 14.539022122762113,
 'lightgbm': 13.367604985812358}